In [67]:
import json
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from sklearn.metrics import classification_report
import wandb
from transformers import BertPreTrainedModel, BertModel, BertConfig
import pickle
from torch.nn.functional import leaky_relu
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

In [68]:
# Source: https://github.com/LCS2-IIITD/Emotion-Flip-Reasoning/blob/main/Dataloaders/nlp_utils.py
import string
import nltk
import re

numbers = {
    "0":"zero",
    "1":"one",
    "2":"two",
    "3":"three",
    "4":"four",
    "5":"five",
    "6":"six",
    "7":"seven",
    "8":"eight",
    "9":"nine"
}

def remove_puntuations(txt):
    punct = set(string.punctuation)
    txt = " ".join(txt.split("."))
    txt = " ".join(txt.split("!"))
    txt = " ".join(txt.split("?"))
    txt = " ".join(txt.split(":"))
    txt = " ".join(txt.split(";"))
    
    txt = "".join(ch for ch in txt if ch not in punct)
    return txt

def number_to_words(txt):
    for k in numbers.keys():
        txt = txt.replace(k,numbers[k]+" ")
    return txt

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'_',' ',text)
    text = number_to_words(text)
    text = remove_puntuations(text)
    text = ''.join([i if ord(i) < 128 else '' for i in text])
    text = ' '.join(text.split())
    return text

In [69]:
train_data = json.load(open('/kaggle/input/semeval3-task-3-dataset/Dataset/ERC_utterance_level/train_utterance_level.json'))
val_data = json.load(open('/kaggle/input/semeval3-task-3-dataset/Dataset/ERC_utterance_level/val_utterance_level.json'))

In [70]:
original_train_data = json.load(open('/kaggle/input/semeval3-task-3-dataset/Dataset/Original_Dataset/Subtask_1_train.json'))

In [71]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [72]:
emotion2int = {
    'anger': 0,
    'joy': 1,
    'fear': 2,
    'disgust': 3,
    'neutral': 4,
    'surprise': 5,
    'sadness': 6
}

In [73]:
L = []
for conversation in original_train_data:
    for utterance in conversation['conversation']:
        L.append(emotion2int[utterance['emotion']])
class_weights=compute_class_weight(class_weight='balanced',classes=list(emotion2int.values()), y=np.array(L))

In [74]:
utterance2vec = pickle.load(open('/kaggle/input/semeval3-task-3-dataset/Dataset/Embeddings/sentence_transformer_utterance2vec_768.pkl', 'rb'))

In [75]:
MAX_CONV_LEN = 35
# Defined index 7 for padding
class ERC_Dataset_Utt_Level(Dataset):
    def __init__(self, data, utterance2vec, device):
        self.data = data
        self.utterance2vec = utterance2vec
        self.device = device

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data[f'id_{idx+1}']['text']
        emotion = self.data[f'id_{idx+1}']['emotion']
        context = self.data[f'id_{idx+1}']['context']

        context_embeddings = [torch.tensor(self.utterance2vec[preprocess_text(utterance)]).to(self.device) for utterance in context]
        target_embedding = torch.tensor(self.utterance2vec[preprocess_text(text)]).to(self.device)
        context_embeddings.append(target_embedding)
                
        if(len(context_embeddings)<MAX_CONV_LEN):
            num_pads = MAX_CONV_LEN - len(context_embeddings)
            attention_mask = [1]*len(context_embeddings) + [0]*num_pads
            context_embeddings = context_embeddings + [torch.zeros(768).to(self.device)]*num_pads
        else:
            context_embeddings = context_embeddings[len(context_embeddings)-MAX_CONV_LEN:]
            attention_mask = [1]*MAX_CONV_LEN

        context_embeddings = torch.stack(context_embeddings)
        attention_mask = torch.tensor(attention_mask)

        return {
            'context_embeddings': context_embeddings,
            'target_embedding': target_embedding,
            'attention_mask': attention_mask,   
            'emotion': emotion2int[emotion]
        }

In [76]:
class BertForSentenceClassificationGivenContext(BertPreTrainedModel):
    def __init__(self, config, weights):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.config = config
        self.bert = BertModel(config)
        classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.weights = weights
        self.dropout = nn.Dropout(classifier_dropout)
        self.classifier = nn.Linear(config.hidden_size*2, config.num_labels)

        self.post_init()

    def forward(self, context_embeds, target_embeds, attention_mask, labels=None):
        output = self.bert(inputs_embeds=context_embeds, attention_mask=attention_mask)
        out_target = [output.last_hidden_state[b][int(sum(attention_mask[b]))-1] for b in range(output.last_hidden_state.shape[0])]
        out_target = torch.stack(out_target)
        out_target = self.dropout(out_target)
        out_target_cat = torch.cat((out_target, target_embeds), 1)
        logits = self.classifier(out_target_cat)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss(weight=self.weights.to(device))
            loss = loss_fct(logits.view(-1, self.num_labels).to(device), labels.view(-1).to(device))

        return {'loss': loss, 'logits': logits}
        

In [77]:
train_dataset = ERC_Dataset_Utt_Level(train_data, utterance2vec, device)
val_dataset = ERC_Dataset_Utt_Level(val_data, utterance2vec, device)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [78]:
config = BertConfig.from_pretrained('bert-base-uncased', num_labels=7)
model = BertForSentenceClassificationGivenContext.from_pretrained('bert-base-uncased', config=config, weights=torch.from_numpy(class_weights).float()).to(device)

Some weights of BertForSentenceClassificationGivenContext were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [79]:
epochs = 30
optimizer = AdamW(model.parameters(), lr=1e-6)

In [80]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_login_key")
wandb.login(key=secret_value_0)

wandb: Currently logged in as: parth-dholariya9221 (TECPEC). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [81]:
wandb.init(project='TECPEC', name='BERT_Utt_Level', config={
    'Embedding': 'Sentence-Transformer',
    'Level': 'Utterance Level',
    'Epochs': epochs,
    'Optimizer': 'AdamW',
    'Learning Rate': 1e-6,
    'Batch Size': 16
})

In [82]:
for epoch in range(epochs):
    model.train()
    train_pred, train_true, train_loss = [], [], 0.0
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        context_embeddings, target_embedding, emotions, attention_mask = batch['context_embeddings'].to(device), batch['target_embedding'].to(device), batch['emotion'].to(device), batch['attention_mask'].to(device)
        outputs = model(context_embeds=context_embeddings, target_embeds=target_embedding, attention_mask=attention_mask, labels=emotions)
        loss = outputs['loss']
        loss.backward()
        optimizer.step()
        train_pred.extend(torch.argmax(outputs['logits'], 1).tolist())
        train_true.extend(emotions.tolist())
        train_loss += loss.item()
    train_loss /= len(train_loader) 
    model.eval()
    val_pred, val_true, val_loss = [], [], 0.0
    with torch.no_grad():
        for batch in tqdm(val_loader):
            context_embeddings, target_embedding, emotions, attention_mask = batch['context_embeddings'].to(device), batch['target_embedding'].to(device), batch['emotion'].to(device), batch['attention_mask'].to(device)
            outputs = model(context_embeds=context_embeddings, target_embeds=target_embedding, attention_mask=attention_mask, labels=emotions)
            loss = outputs['loss']
            val_pred.extend(torch.argmax(outputs['logits'], 1).tolist())
            val_true.extend(emotions.tolist())
            val_loss += loss.item()
            
    val_loss /= len(val_loader)
    train_report = classification_report(train_true, train_pred, target_names=emotion2int.keys(), zero_division=0)
    val_report = classification_report(val_true, val_pred, target_names=emotion2int.keys(), zero_division=0)

    train_report_dict = classification_report(train_true, train_pred, target_names=emotion2int.keys(), output_dict=True, zero_division=0)
    val_report_dict = classification_report(val_true, val_pred, target_names=emotion2int.keys(), output_dict=True, zero_division=0)
    wandb.log({
        'train_loss': train_loss,
        'val_loss': val_loss,
        'train_accuracy': train_report_dict['accuracy'],
        'val_accuracy': val_report_dict['accuracy'],
        'Macro train_f1': train_report_dict['macro avg']['f1-score'],
        'Macro val_f1': val_report_dict['macro avg']['f1-score'],
        'Weighted train_f1': train_report_dict['weighted avg']['f1-score'],
        'Weighted val_f1': val_report_dict['weighted avg']['f1-score'],
    })
    print(f"Epoch: {epoch+1}, Train Loss: {train_loss}, Val Loss: {val_loss}")
    print(f"Train Report: \n{train_report}")
    print(f"Val Report: \n{val_report}")


100%|██████████| 93/93 [00:03<00:00, 24.58it/s]


Epoch: 1, Train Loss: 1.950147879610577, Val Loss: 1.9272488150545346
Train Report: 
              precision    recall  f1-score   support

       anger       0.13      0.14      0.14      1423
         joy       0.16      0.21      0.18      2047
        fear       0.03      0.04      0.03       336
     disgust       0.02      0.01      0.01       372
     neutral       0.44      0.31      0.37      5299
    surprise       0.15      0.18      0.16      1656
     sadness       0.10      0.15      0.12      1011

    accuracy                           0.23     12144
   macro avg       0.15      0.15      0.14     12144
weighted avg       0.26      0.23      0.24     12144

Val Report: 
              precision    recall  f1-score   support

       anger       0.14      0.20      0.17       192
         joy       0.27      0.26      0.27       254
        fear       0.00      0.00      0.00        37
     disgust       0.04      0.10      0.06        42
     neutral       0.46      0.26 

100%|██████████| 93/93 [00:03<00:00, 24.04it/s]


Epoch: 2, Train Loss: 1.9347428466648608, Val Loss: 1.9189553619712911
Train Report: 
              precision    recall  f1-score   support

       anger       0.14      0.21      0.17      1423
         joy       0.20      0.24      0.22      2047
        fear       0.04      0.01      0.02       336
     disgust       0.05      0.05      0.05       372
     neutral       0.46      0.30      0.36      5299
    surprise       0.16      0.20      0.18      1656
     sadness       0.10      0.16      0.12      1011

    accuracy                           0.24     12144
   macro avg       0.16      0.17      0.16     12144
weighted avg       0.28      0.24      0.25     12144

Val Report: 
              precision    recall  f1-score   support

       anger       0.17      0.38      0.23       192
         joy       0.25      0.34      0.29       254
        fear       0.14      0.03      0.05        37
     disgust       0.03      0.07      0.05        42
     neutral       0.43      0.21

100%|██████████| 93/93 [00:03<00:00, 24.18it/s]


...

Epoch: 28, Train Loss: 1.4030019496426438, Val Loss: 1.9594216064740253
Train Report: 
              precision    recall  f1-score   support

       anger       0.36      0.41      0.38      1423
         joy       0.39      0.52      0.45      2047
        fear       0.22      0.62      0.32       336
     disgust       0.21      0.63      0.32       372
     neutral       0.65      0.29      0.40      5299
    surprise       0.35      0.36      0.36      1656
     sadness       0.32      0.53      0.40      1011

    accuracy                           0.39     12144
   macro avg       0.36      0.48      0.38     12144
weighted avg       0.48      0.39      0.40     12144

Val Report: 
              precision    recall  f1-score   support

       anger       0.26      0.26      0.26       192
         joy       0.45      0.47      0.46       254
        fear       0.04      0.35      0.07        37
     disgust       0.09      0.17      0.12        42
     neutral       0.58      0.2

100%|██████████| 93/93 [00:03<00:00, 24.20it/s]


Epoch: 29, Train Loss: 1.3909649617273032, Val Loss: 1.918106238047282
Train Report: 
              precision    recall  f1-score   support

       anger       0.37      0.45      0.41      1423
         joy       0.40      0.53      0.46      2047
        fear       0.21      0.63      0.31       336
     disgust       0.22      0.66      0.34       372
     neutral       0.66      0.29      0.40      5299
    surprise       0.36      0.37      0.36      1656
     sadness       0.33      0.52      0.41      1011

    accuracy                           0.40     12144
   macro avg       0.37      0.49      0.38     12144
weighted avg       0.49      0.40      0.40     12144

Val Report: 
              precision    recall  f1-score   support

       anger       0.28      0.27      0.27       192
         joy       0.40      0.52      0.46       254
        fear       0.05      0.22      0.09        37
     disgust       0.08      0.26      0.12        42
     neutral       0.57      0.24

100%|██████████| 93/93 [00:03<00:00, 24.21it/s]

Epoch: 30, Train Loss: 1.372480718985848, Val Loss: 1.9485335055217947
Train Report: 
              precision    recall  f1-score   support

       anger       0.38      0.46      0.42      1423
         joy       0.40      0.52      0.45      2047
        fear       0.22      0.65      0.32       336
     disgust       0.22      0.63      0.33       372
     neutral       0.66      0.29      0.40      5299
    surprise       0.35      0.37      0.36      1656
     sadness       0.33      0.54      0.41      1011

    accuracy                           0.40     12144
   macro avg       0.37      0.49      0.38     12144
weighted avg       0.49      0.40      0.40     12144

Val Report: 
              precision    recall  f1-score   support

       anger       0.26      0.27      0.27       192
         joy       0.35      0.61      0.45       254
        fear       0.07      0.16      0.10        37
     disgust       0.08      0.24      0.12        42
     neutral       0.56      0.21

In [83]:
wandb.finish()

Macro train_f1,▁▁▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇███
Macro val_f1,▁▂▂▁▄▂▄▃▅▅▅▅▄▆▆▅▆▇▇▇▇▆▇▇▇▇▇▇██
Weighted train_f1,▁▁▂▂▃▃▃▃▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
Weighted val_f1,▃▂▃▁▅▁▃▁▅▄▄▄▃▅▅▄▄▇▆▆▇▅▇▇▇▆▇██▇
train_accuracy,▁▁▂▂▃▃▃▃▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train_loss,███▇▇▇▇▆▆▆▅▅▅▅▅▄▄▄▃▃▃▃▃▂▂▂▂▁▁▁
val_accuracy,▁▂▄▃▅▁▃▁▅▄▄▄▂▅▄▄▅▇▆▆▅▅▇▆▇▇▇▇██
val_loss,▆▆▅▄▃▃▂▂▁▂▂▂▃▂▂▃▃▃▄▄▄▄▄▅▅▆▇█▆▇
Macro train_f1,0.38476
Macro val_f1,0.26761
Weighted train_f1,0.40351


In [84]:
torch.save(model, '/kaggle/working/BERT_Utt_Level.pth')